In [69]:
import fitz
from collections import Counter

In [17]:
pdf_address = "li.pdf"
#pdf_address = "judea.pdf"
pdf_doc = fitz.open(pdf_address)

In [23]:
page = pdf_doc.load_page(10)
page_text = page.get_text("text")
page_text

'1 Introduction\n3\nelectrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is one key point to facilitate functional imaging with\nalive animals. For example, with a cranial window over visual cortex of a transgenic\nmouse, it is able to image large scale of neu

In [28]:
page_blocks = page.get_text("blocks")
page_blocks


[(113.38600158691406,
  46.6934814453125,
  510.26995849609375,
  62.151676177978516,
  '1 Introduction\n3\n',
  0,
  0),
 (113.00399780273438,
  82.85345458984375,
  512.27734375,
  293.4193115234375,
  'electrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is

In [43]:
page_dicts = page.get_text("dict")

In [44]:
page_dicts

{'width': 595.276,
 'height': 841.89,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (113.38600158691406,
    46.6934814453125,
    510.26995849609375,
    62.151676177978516),
   'lines': [{'spans': [{'size': 10.909099578857422,
       'flags': 4,
       'font': 'LMRoman10-Regular',
       'color': 0,
       'ascender': 1.1269999742507935,
       'descender': -0.28999999165534973,
       'text': '1 Introduction',
       'origin': (113.38600158691406, 58.988037109375),
       'bbox': (113.38600158691406,
        46.6934814453125,
        184.05514526367188,
        62.151676177978516)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (113.38600158691406,
      46.6934814453125,
      184.05514526367188,
      62.151676177978516)},
    {'spans': [{'size': 10.909099578857422,
       'flags': 4,
       'font': 'LMRoman10-Regular',
       'color': 0,
       'ascender': 1.1269999742507935,
       'descender': -0.28999999165534973,
       'text': '3',
       'origin': (504.8153991

In [52]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

In [64]:
blocks = page.get_text("dict", flags=11)["blocks"]
page_font_sizes = []
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            print("")
            page_font_sizes.append(s["size"])
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            print("Text: '%s'" % s["text"])  # simple print of text
            print(font_properties)


Text: '1Introduction'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: '3'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'electrophysiologytocompensatethelowtemporalresolution.Figure2isanexam-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ple.Intra-cellularrecordingdirectlymeasuresneuronalactivities,usuallytakenas'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'groundtruth.Calciumimagingrecordsfluorescencesignalsasanindicatorofneu-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ronalspikingactivity,whichisindirect.Fluorescencesignalsandneuronalspikes'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'differfromeachother.Figure3showssingleactionpotentialanditsevokedcalcium'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000



In [71]:
page_font_sizes_counter = Counter(page_font_sizes)
page_font_sizes_counter

Counter({10.909099578857422: 29, 11.9552001953125: 1})

In [79]:
page_font_sizes_counter.most_common(1)[0]

(10.909099578857422, 29)

In [78]:
page_font_sizes_counter.most_common(1)

[(10.909099578857422, 29)]

In [83]:
def find_titles(page):
    blocks = page.get_text("dict", flags=11)["blocks"]
    page_font_sizes = []
    page_font_names = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                print("")
                page_font_sizes.append(s["size"])
                page_font_names.append(s["font"])
                
    page_font_sizes_counter = Counter(page_font_sizes)
    most_common_font_size, most_common_count = page_font_sizes_counter.most_common(1)[0]

    if most_common_count / len(page_font_sizes) > 0.5:
        title_font_sizes = [x for x in page_font_sizes if x > most_common_font_size]
    else:
        return None
    
    title_blocks = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                if s["size"] in title_font_sizes:
                    title_blocks.append(b)

    return title_blocks
    
    

In [86]:
find_titles(page)

[{'number': 3,
  'type': 0,
  'bbox': (113.38597106933594,
   527.8118286132812,
   406.64715576171875,
   544.9197387695312),
  'lines': [{'spans': [{'size': 11.9552001953125,
      'flags': 20,
      'font': 'LMSans10-Bold',
      'color': 0,
      'ascender': 1.1339999437332153,
      'descender': -0.296999990940094,
      'text': '1.1.1Intracellularcalciumdynamicsanditsmodeling',
      'origin': (113.38597106933594, 541.3690185546875),
      'bbox': (113.38597106933594,
       527.8118286132812,
       406.64715576171875,
       544.9197387695312)}],
    'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (113.38597106933594,
     527.8118286132812,
     406.64715576171875,
     544.9197387695312)}]}]